In [26]:
import pandas as pd

vgscore_data = pd.read_csv("vgscore.csv")

vgscore_data

,name,platform,r-date,score,user score,developer,genre,players,critics,users
0,The Legend of Zelda: Ocarina of Time,Nintendo64,"November 23, 1998",99,9.1,Nintendo,"Action Adventure,Fantasy",1 Player,22,5749
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4,NeversoftEntertainment,"Sports,Alternative,Skateboarding",1-2,19,647
2,Grand Theft Auto IV,PlayStation3,"April 29, 2008",98,7.6,RockstarNorth,"Action Adventure,Modern,Modern,Open-World",1 Player,64,3806
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.5,Namco,"Action,Fighting,3D",1-2,24,324
4,Grand Theft Auto IV,Xbox360,"April 29, 2008",98,7.9,RockstarNorth,"Action Adventure,Modern,Modern,Open-World",1 Player,86,3364
...,...,...,...,...,...,...,...,...,...,...
17939,Vroom in the Night Sky,Switch,"April 5, 2017",17,3.1,Poisoft,"Sports,Individual,Biking",No Online Multiplayer,15,105
17940,Leisure Suit Larry: Box Office Bust,PlayStation3,"May 5, 2009",17,1.9,Team17,"Action Adventure,Adventure,Third-Person,Open-W...",No Online Multiplayer,11,45
17941,Yaris,Xbox360,"October 10, 2007",17,4.3,BackboneEntertainment,"Driving,Racing,Arcade,Arcade,Automobile",2 Online,7,129
17942,Ride to Hell: Retribution,PC,"June 24, 2013",16,1.3,"Eutechnyx,DeepSilver","Driving,Modern,Racing,Motorcycle,Motocross,Mod...",No info,9,581


In [27]:
vgscore_data.isnull().any()

name          False
platform      False
r-date        False
score         False
user score    False
developer     False
genre         False
players        True
critics       False
users         False
dtype: bool

In [28]:
vgscore_data = vgscore_data.drop(columns=["players"])
vgscore_data.dtypes

name          object
platform      object
r-date        object
score          int64
user score    object
developer     object
genre         object
critics        int64
users          int64
dtype: object

In [29]:
def is_not_ascii(value):
    if isinstance(value, str):
        return any(ord(s) >= 128 for s in value)
    return False

for col in vgscore_data.columns.tolist():
    non_ascii_rows = vgscore_data[vgscore_data[col].apply(is_not_ascii)]
    if not non_ascii_rows.empty:
        print(f"Column: {col}")
        print(non_ascii_rows)

In [30]:
from sklearn.model_selection import train_test_split

X = vgscore_data.drop(columns=['score'])
y = vgscore_data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14355, 8), (3589, 8), (14355,), (3589,))

In [31]:
import catboost as cat

cf = ['name', 'platform', 'r-date', 'user score', 'developer', 'genre', 'critics', 'users']
model = cat.CatBoostRegressor(random_state=100,cat_features=cf,verbose=0)
model.fit(X_train, y_train)

In [32]:
from sklearn.metrics import root_mean_squared_error

predictions = model.predict(X_test)
score = root_mean_squared_error(y_test, predictions)

score

7.991302168154645